In [29]:
# https://colab.research.google.com/github/pytorch/vision/blob/temp-tutorial/tutorials/torchvision_finetuning_instance_segmentation.ipynb
import os
def rt(p):
    return os.path.join('../../',p)

In [30]:
from PIL import Image
img = Image.open(rt('data/Hackathon/Folder A  7 x same headstamp/17 88 image A1.jpg'))

In [31]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image



In [32]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [33]:
#%%shell

# Download TorchVision repo to use some files from
# references/detection
#git clone https://github.com/pytorch/vision.git
#cd vision
#git checkout v0.3.0

#cp references/detection/utils.py ../
#cp references/detection/transforms.py ../
#cp references/detection/coco_eval.py ../
#cp references/detection/engine.py ../
#cp references/detection/coco_utils.py ../

In [34]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [39]:
# rsim: TODO here...
# use our dataset and defined transformations
from coco_utils import CocoDetection, ConvertCocoPolysToMask

dataset = CocoDetection(rt('data/dataset'), rt('data/dataset/coco.json'), T.Compose([ConvertCocoPolysToMask(),get_transform(train=True)]))

#dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = CocoDetection(rt('data/dataset'), rt('data/dataset/coco.json'), T.Compose([ConvertCocoPolysToMask(),get_transform(train=False)]))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
#dataset = torch.utils.data.Subset(dataset, indices[:-50])
#dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [40]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has three classes: background, casing, and primer
num_classes = 3

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [42]:
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [ 0/10]  eta: 0:08:53  lr: 0.000560  loss: 6.3345 (6.3345)  loss_classifier: 0.5436 (0.5436)  loss_box_reg: 0.0200 (0.0200)  loss_mask: 1.0446 (1.0446)  loss_objectness: 2.9900 (2.9900)  loss_rpn_box_reg: 1.7363 (1.7363)  time: 53.3354  data: 10.0440  max mem: 0
Epoch: [0]  [ 9/10]  eta: 0:00:29  lr: 0.005000  loss: 1.8946 (2.7446)  loss_classifier: 0.2078 (0.2611)  loss_box_reg: 0.0034 (0.0081)  loss_mask: 0.3208 (0.4598)  loss_objectness: 0.4681 (0.9996)  loss_rpn_box_reg: 0.7255 (1.0160)  time: 29.8928  data: 1.0078  max mem: 0
Epoch: [0] Total time: 0:04:59 (29.9291 s / it)


AssertionError: Torch not compiled with CUDA enabled